In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import  statsmodels 
import matplotlib.pyplot as plt



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Here we start to analyze all our datasets to know: 
* Shape
* NaN values
* type of data
* descripcion

among others


In [ ]:
dataframes = {}

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        name_without_extension = os.path.splitext(filename)[0]
        df = pd.read_csv(os.path.join(dirname, filename))
        dataframes[name_without_extension] = df

for key,value in dataframes.items():
    print('\n \n \n ',key)
    print('\n Descripcion\n ',value.describe())
    print('\n tipos\n ',value.dtypes)
    print('\n Dataframe\n', value.head(5))
    
for key,value in dataframes.items():
     
    print('\n \n \n ',key,value.columns)


Hands on our dataframes to clean and transform some data.


In [ ]:
df_holidays= dataframes['holidays_events']
df_holidays_false = df_holidays[df_holidays['transferred'] == False]
df_holidays_false.head(5)



In [ ]:
transactions_stores = pd.merge(dataframes['transactions'],dataframes['stores'], on='store_nbr', how='left')
transactions_stores_oil = pd.merge(transactions_stores,dataframes['oil'], on='date', how='left')

In [ ]:
transactions_stores_oil['holiday'] = 0
df_holidays_false.shape

for i in range(df_holidays_false.shape[0]):

    if df_holidays_false['locale'].iloc[i] == 'National':
        
        date_to_match = df_holidays_false['date'].iloc[i]
        transactions_stores_oil.loc[transactions_stores_oil['date'] == date_to_match, 'holiday'] = 1
        
    elif df_holidays_false['locale'].iloc[i] == 'Regional':

        date_to_match = df_holidays_false['date'].iloc[i]
        state_to_match = df_holidays_false['locale_name'].iloc[i]
        transactions_stores_oil.loc[(transactions_stores_oil['date'] == date_to_match) & (transactions_stores_oil['state'] == state_to_match), 'holiday'] = 1
    
    elif df_holidays_false['locale'].iloc[i] == 'Local':
        date_to_match = df_holidays_false['date'].iloc[i]
        city_to_match = df_holidays_false['locale_name'].iloc[i]
        transactions_stores_oil.loc[(transactions_stores_oil['date'] == date_to_match) & (transactions_stores_oil['city'] == city_to_match), 'holiday'] = 1

In [ ]:
tsoh = transactions_stores_oil.copy()

for i in range(tsoh.shape[0]-1):
    
    if np.isnan(tsoh['dcoilwtico'][i]):
        if np.isnan(tsoh['dcoilwtico'][i + 1]):
            window_size = 50
            lower_bound = max(0, i - window_size)
            upper_bound = min(tsoh.shape[0], i + window_size + 1)
            mean_value = np.nanmean(tsoh.loc[lower_bound:upper_bound, 'dcoilwtico'])
            tsoh.loc[i, 'dcoilwtico'] = mean_value
        else:
            tsoh.loc[i, 'dcoilwtico'] = tsoh['dcoilwtico'][i + 1]

print(tsoh['dcoilwtico'].isnull().sum())

Here starts our correlation analysis
Our hypotesis are:
*  *Correlation between transaction/sales and Oil price*
*  *Correlation between transaction/sales and holidays*
*  *Correlation between transaction/sales and days*

In [ ]:
tsoh

In [ ]:
date_group = tsoh.groupby(by= 'date').agg({'dcoilwtico': 'first', 'holiday': 'first', 'transactions': 'sum'})

date_group_df =pd.DataFrame(date_group)
date_group_df

* Visualizar las ventas crecimiento 
* visualizar los 10 dias del anio con mas ventas
* visualizar con lineas como varian dependiendo el mes
* ventas por estados 
* ventas por estado y meses 
* localidades con mas ventas
* tiendas con mas ventas y clusters con mas ventas 

In [ ]:
sns.set(style="whitegrid") 
plt.figure(figsize=(10, 6))  


sns.lineplot(data=df, x="date", y="transactions")

# Personaliza los ejes y el título
plt.xlabel("Fecha")
plt.ylabel("Transacciones")
plt.title("Transacciones a lo largo del tiempo")

# Rota las etiquetas del eje x para una mejor legibilidad
plt.xticks(rotation=45)

plt.tight_layout()  # Ajusta la disposición de los elementos
plt.show()


In [ ]:
tsoh['date'] = pd.to_datetime(tsoh['date'])  # Asegúrate de que la columna 'date' sea de tipo datetime
tsoh['year'] = tsoh['date'].dt.year
tsoh['month'] = tsoh['date'].dt.month

monthly_data = tsoh.groupby(['year', 'month'])['transactions'].sum().reset_index()



In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

sns.lineplot(data=monthly_data, x='month', y='transactions', hue='year', marker='o')

plt.xlabel("Meses")
plt.ylabel("Transacciones")
plt.title("Transacciones por Mes y Año")

plt.xticks(range(1, 13), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
plt.legend(title="Año")
plt.tight_layout()
plt.show()


* ventas por estados 
* ventas por estado y meses 

In [ ]:
tsoh['date'] = pd.to_datetime(tsoh['date'])  # Asegúrate de que la columna 'date' sea de tipo datetime
tsoh['month'] = tsoh['date'].dt.month

month_state_data = tsoh.groupby(['state', 'month'])['transactions'].sum().reset_index()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

sns.lineplot(data=month_state_data, x='month', y='transactions', hue='state', marker='o')

plt.xlabel("Meses")
plt.ylabel("Transacciones")
plt.title("Transacciones por estado y Año")

plt.xticks(range(1, 13), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
plt.legend(title="Estado",bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:

state_monthly_data = tsoh.groupby(['state', 'month'])['transactions'].sum().reset_index()
state_total_transactions = tsoh.groupby('state')['transactions'].sum()
state_monthly_data['transactions_percentage'] = (
    state_monthly_data.apply(lambda row: row['transactions'] / state_total_transactions[row['state']], axis=1)
)


sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

plot = sns.lineplot(data=state_monthly_data, x='month', y='transactions_percentage', hue='state', marker='o')

plt.xlabel("Meses")
plt.ylabel("Porcentaje de Transacciones")
plt.title("Porcentaje de Transacciones por Mes y Estado")

plt.xticks(range(1, 13), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])

plt.legend(title="Estado", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


Podemos apreciar los outliers provocados por el terremoto en el mes de septiembre donde las ventas cayeron de manera notable, por lo que estos datos no seran tenidos en cuenta

* localidades con mas ventas
* tiendas con mas ventas y clusters con mas ventas

In [ ]:
stores_sales = tsoh.groupby(by = ['store_nbr','cluster'])['transactions'].sum().reset_index()

stores_sales = stores_sales.sort_values(by= 'transactions', ascending = False)


In [ ]:
total_sum = stores_sales['transactions'].sum()
stores_sales['total_percent']= 0

stores_sales['transactions_percentage'] = (
    stores_sales.apply(lambda row: round(((row['transactions'] / total_sum)*100),2), axis=1)
)

stores_sales['transactions_percentage_total'] = 100- stores_sales['transactions_percentage'].cumsum()

stores_sales = stores_sales.sort_values(by= 'transactions_percentage', ascending = False)   
stores_sales.head(10)

In [ ]:

plt.figure(figsize=(12, 6))
plt.subplot(2, 2, 1)  # Subplot 1


sns.barplot(data=stores_sales, x='store_nbr', y='transactions_percentage')
plt.xlabel('Número de Tienda')
plt.ylabel('Porcentaje de Transacciones')
plt.title('Porcentaje de Transacciones por Tienda')
plt.xticks(rotation=90, fontsize= 8)


plt.subplot(2, 2, 2)  # Subplot 2


sns.barplot(data=stores_sales, x='store_nbr', y='transactions_percentage_total',order=stores_sales['store_nbr'])
plt.xlabel('Número de Tienda')
plt.ylabel('Porcentaje Total de Transacciones')
plt.title('Porcentaje Total de Transacciones por Tienda')
plt.xticks(rotation=90, fontsize= 8)



plt.subplot(2, 2, 3)  # Subplot 2


sns.barplot(data=stores_sales, x='cluster', y='transactions_percentage')
plt.xlabel('Número de Tienda')
plt.ylabel('Porcentaje Total de Transacciones')
plt.title('Porcentaje Total de Transacciones por Tienda')
plt.xticks(rotation=90, fontsize= 8)


plt.subplot(2, 2, 4)  # Subplot 2


sns.barplot(data=stores_sales, x='cluster', y='transactions_percentage_total',order=stores_sales['cluster'])
plt.xlabel('Número de Tienda')
plt.ylabel('Porcentaje Total de Transacciones')
plt.title('Porcentaje Total de Transacciones por Tienda')
plt.xticks(rotation=90, fontsize= 8)

plt.tight_layout()  # Ajustar la disposición de los subplots
plt.show()



In [ ]:

corr_matrix= tsoh.corr(numeric_only=True)


plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación')
plt.show()

Ok, luego de corroborar que los datos se comportan de manera similar y que no existe correlacion fuerte entre el precio del petroleo y las vacaciones/feriados. De hecho, no existe correlacion fuerte entre ninguna variable que merezca especial atencion. Por lo que procederemos a realizar los modelos solo con los datos proporcionados por los dataframe Train, Test, Cluster, tienda y fecha.

Avanzaremos con los modelos de ML:
1. Para este problema podemos usar una simple regresion por el tipo de dato que tenemos. Por su simpleza sera nuestra primera opcion.
2. Como un modelo un poco mas avanzado tambien desarrollaremos Regresión de Árboles de Decisión: Random Forest y Gradient Boosting
3. Evaluaremos los resultados para ver como se desempeñaron los modelos 



In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df= tsoh.copy()

In [ ]:
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6])
df['day_of_fortnight'] = df['date'].dt.day - 1  

df.head(10)


In [ ]:
train= dataframes['train']
train['date'] = pd.to_datetime(train['date'])
train.dtypes

In [ ]:
df = df.drop(labels= ['transactions','city','state','type','dcoilwtico', 'holiday','month'], axis = 1)

In [ ]:
print(df.tail(10),train.tail(10))

In [ ]:
dataframe_to_use = pd.merge(train,df,on=['date','store_nbr'], how= 'inner')

In [ ]:
print(dataframe_to_use.shape,train.shape,df.shape)
dataframe_to_use.tail(10)

We are going to take the last month (15-07 to 15-08) to do our evaluation, while we use the last year before the evaluation period to train our data. 
Also, taking this time we avoided to take in consideration some outliers provoked by the earthquake.

Now, you can see 

In [ ]:
x_train = dataframe_to_use[(dataframe_to_use['date'] > '2016-07-15') & (dataframe_to_use['date'] <'2017-07-15')].drop(labels=['sales','date'], axis=1)
y_train = dataframe_to_use[(dataframe_to_use['date'] > '2016-07-15') & (dataframe_to_use['date'] < '2017-07-15')]['sales']
x_test = dataframe_to_use[dataframe_to_use['date'] > '2017-07-15'].drop(labels=['sales','date'], axis=1)
y_test = dataframe_to_use[dataframe_to_use['date'] > '2017-07-15']['sales']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Update SciPy to the version which is compatible with Numpy

In [ ]:
ct = ColumnTransformer(transformers=[('ohe',OneHotEncoder(sparse_output=False),['family','is_weekend'])],
                       remainder='passthrough',
                       n_jobs=-1,verbose_feature_names_out=False)


X_train = ct.fit_transform(x_train)
X_train = pd.DataFrame(X_train, columns=ct.get_feature_names_out())
X_test = ct.transform(x_test)  # Usar transform en lugar de fit_transform
X_test = pd.DataFrame(X_test, columns=ct.get_feature_names_out())


In [ ]:
print(X_train.dtypes,'\n \n TEST \n \n',X_test.dtypes,'\n \n TRAIN \n \n' ,y_train.dtypes,'\n \n TRAIN TEST \n \n', y_test.dtypes )
nan_count_train = X_train.isna().sum()
nan_count_test = X_test.isna().sum()
nan_count_trainy = y_train.isna().sum()
nan_count_testy= y_test.isna().sum()
print(nan_count_train,'\n \n NAN XTEST \n \n',nan_count_test,'\n \n NAN Y TRAIN \n \n',nan_count_trainy,'\n \n NAN T TEST \n \n',nan_count_testy)


In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=200,n_jobs=-1, eval_metric='rmse', early_stopping_rounds=5, random_state=42)
xgb_model.fit(X=X_train,y=y_train,eval_set=[(X_test,y_test)],verbose=10)

Verificamos que los valores sean mayores o iguales a 0.

In [ ]:
y_pred = xgb_model.predict(X_test)
num_negativos = np.sum(y_pred < 0)
num_negativos

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

linear_predictions = linear_model.predict(X_test)

linear_mse = mean_squared_error(y_test, linear_predictions)
linear_rmse = np.sqrt(linear_mse)
print("Regresión Lineal RMSE:", round(linear_rmse,2))
num_negativos = np.sum(linear_predictions < 0)
print(num_negativos)

Tomando en consideracion ambos RSME (Root mean squared error) el mejor modelo es XGBoost ya que tienen valores mas bajos. Ahora bien, separaremos para cada una de las familias para ver de manera visual como se comporta en cada una de ellas y verificar la posible existencia de errores. 
Ya de por si, antes de comenzar podemos conocer que en ambos modelos tenemos una elevada cantidad de valores inferiores a Cero lo cual es a priori incorrecto.

In [ ]:
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Realiza predicciones con el modelo de árbol de decisión
tree_predictions = tree_model.predict(X_test)

# Calcula el error cuadrático medio
tree_mse = mean_squared_error(y_test, tree_predictions)
tree_rmse = np.sqrt(tree_mse)
print("Árbol de Decisión RMSE:", round(tree_rmse, 2))

# Calcula el número de valores negativos en las predicciones
num_negativos_tree = np.sum(tree_predictions < 0)
print(num_negativos_tree)


In [ ]:
x_test['y_pred']=y_pred
x_test['linear']=linear_predictions
x_test['y_test']= y_test
x_test['tree']= tree_predictions
x_test.head(10)

In [ ]:
families = x_test['family'].unique()


num_subplots = len(families)


plt.figure(figsize=(12, 8*num_subplots))


colores = {'y_pred': 'red', 'linear': 'orange','tree': 'pink', 'y_test': 'blue'}
estilos = {'y_pred': '--', 'linear': '--', 'tree': '-.','y_test': '-'}


for i, family in enumerate(families):
    familia_df = x_test[x_test['family'] == family]
    plt.subplot(num_subplots, 1, i + 1)
    

    for variable, color, estilo in zip(['y_pred', 'linear','tree', 'y_test'], colores.values(), estilos.values()):
        plt.plot(familia_df['id'], familia_df[variable], label=f'{variable}', color=color, linestyle=estilo)
    

    plt.title(f'Familia: {family}')
    plt.legend()


plt.tight_layout()


plt.show()

Podemos apreciar rapidamente que la opcion de regresion lineal se ajusta bastante mal al modelo real por lo que descartaremos de los graficos para apreciar mejor a XGBoost

In [ ]:
families = x_test['family'].unique()


num_subplots = len(families)


plt.figure(figsize=(12, 8*num_subplots))


colores = {'y_pred': 'red', 'y_test': 'blue'}
estilos = {'y_pred': '--',  'y_test': '-'}


for i, family in enumerate(families):
    familia_df = x_test[x_test['family'] == family]
    plt.subplot(num_subplots, 1, i + 1)
    

    for variable, color, estilo in zip(['y_pred', 'y_test'], colores.values(), estilos.values()):
        plt.plot(familia_df['id'], familia_df[variable], label=f'{variable}', color=color, linestyle=estilo)
    

    plt.title(f'Familia: {family}')
    plt.legend()


plt.tight_layout()


plt.show()

In [ ]:
families = x_test['family'].unique()


num_subplots = len(families)


plt.figure(figsize=(12, 8*num_subplots))


colores = {'tree': 'pink', 'y_test': 'blue'}
estilos = {'tree': '-.',  'y_test': '-'}


for i, family in enumerate(families):
    familia_df = x_test[x_test['family'] == family]
    plt.subplot(num_subplots, 1, i + 1)
    

    for variable, color, estilo in zip(['tree', 'y_test'], colores.values(), estilos.values()):
        plt.plot(familia_df['id'], familia_df[variable], label=f'{variable}', color=color, linestyle=estilo)
    

    plt.title(f'Familia: {family}')
    plt.legend()


plt.tight_layout()


plt.show()